<a href="https://colab.research.google.com/github/Sushmitha-Katti/EVA-4/blob/master/Session12/S12_V4(Reduce_Overfitting_abd_increase_accuracy)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object classification on Tiny Imagenet Data

In [1]:
!pip install --no-cache-dir git+https://github.com/albu/albumentations > /dev/null && echo "Albumentation library is successfully installed!"
!git clone https://github.com/Sushmitha-Katti/PyTNet 
import sys
sys.path.append("PyTNet")

  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-rbv0yt9c
Albumentation library is successfully installed!
fatal: destination path 'PyTNet' already exists and is not an empty directory.


## **Importing Required files**

In [0]:
import torch
import train_test_loader
import show_images
import train_test
import evaluate
from Albumentationtransform import AlbumentationTransforms
from GradCam import GradCAM,visualize_cam
from LR_Range_Test import LR_test
from tinyimagenet import TinyImageNetDataSet

In [3]:
import albumentations as A
import cv2
channel_means = (0.5, 0.5, 0.5)
channel_stdevs = (0.5, 0.5, 0.5)
train_transform = AlbumentationTransforms([       
                                                                
                                      A.HorizontalFlip(p = 0.7),
                                      A.PadIfNeeded(min_height=70, min_width=70, border_mode=4, value=None, mask_value=None, always_apply=False, p=1.0),
                                      A.GaussNoise(var_limit=(10.0, 50.0), mean=0, always_apply=False, p=0.5),
                                      A.RandomCrop(64, 64, always_apply=False, p=1.0),
                                      A.VerticalFlip(always_apply=False, p=0.5),
                                      A.Rotate(limit=50, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
                                      A.Normalize(mean=channel_means, std=channel_stdevs),
                                      A.Cutout(num_holes=1, max_h_size=32,max_w_size = 32,p=0.7) 
                                       ])
test_transform = AlbumentationTransforms([A.Normalize(mean=channel_means, std=channel_stdevs)])
train_dataset , test_dataset,classes = TinyImageNetDataSet(train_split = 70,test_transforms = test_transform,train_transforms = train_transform)

Images already downloaded...


## **DataLoader**

In [0]:
trainloader, testloader = train_test_loader.load(train_dataset,test_dataset,batch_size=640)

## **Used Torchvisions resnet model, but not pre trained. Print the summary**

In [13]:
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = torchvision.models.resnet18(pretrained=False, num_classes=200).to(device)
summary(model, input_size=(3, 64, 64))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,408
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,864
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
       BasicBlock-11           [-1, 64, 16, 16]               0
           Conv2d-12           [-1, 64, 16, 16]          36,864
      BatchNorm2d-13           [-1, 64, 16, 16]             128
             ReLU-14           [-1, 64,

## **Initialise Optimiser,Scheduler, Criterion**

In [0]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR,StepLR

optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9,weight_decay = 0.0001,nesterov = True ) 
criterion = nn.CrossEntropyLoss()
scheduler = OneCycleLR(optimizer, max_lr = 0.2, total_steps=None, epochs=30, steps_per_epoch=1, pct_start=1/3, anneal_strategy='linear', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=10.0,final_div_factor =10)


## **Train Model**

In [15]:

train_test.train_model(model,device,trainloader,testloader,optimizer,criterion,30,scheduler,batch_scheduler=False,best_acc = 0,path = "tinymodel.pt")

  0%|          | 0/121 [00:00<?, ?it/s]

EPOCH: 1 LR: 0.02


Loss=4.696480751037598 Batch_id=120 Accuracy=2.94: 100%|██████████| 121/121 [00:54<00:00,  2.24it/s]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0074, Accuracy: 1951/33000 (5.91%)

accuracy increased, Saving model....
EPOCH: 2 LR: 0.04


Loss=4.178760528564453 Batch_id=120 Accuracy=8.55: 100%|██████████| 121/121 [00:54<00:00,  2.24it/s]
  0%|          | 0/121 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0043, Accuracy: 12778/33000 (38.72%)

EPOCH: 29 LR: 0.011900000000000022


Loss=1.8239244222640991 Batch_id=120 Accuracy=56.74: 100%|██████████| 121/121 [00:53<00:00,  2.27it/s]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0042, Accuracy: 13295/33000 (40.29%)

accuracy increased, Saving model....
EPOCH: 30 LR: 0.0020000000000000018


Loss=1.8457741737365723 Batch_id=120 Accuracy=59.56: 100%|██████████| 121/121 [00:53<00:00,  2.24it/s]
100%|██████████| 52/52 [00:05<00:00,  9.03it/s]



Test set: Average loss: 0.0041, Accuracy: 13635/33000 (41.32%)

accuracy increased, Saving model....


In [16]:
scheduler = OneCycleLR(optimizer, max_lr = 0.02, total_steps=None, epochs=10, steps_per_epoch=1, pct_start=1/2, anneal_strategy='linear', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=10.0,final_div_factor =2)

train_test.train_model(model,device,trainloader,testloader,optimizer,criterion,10,scheduler,batch_scheduler=False,best_acc = 41.32,path = "tinymodel2.pt")

  0%|          | 0/121 [00:00<?, ?it/s]

EPOCH: 1 LR: 0.002


Loss=1.7093144655227661 Batch_id=120 Accuracy=60.34: 100%|██████████| 121/121 [00:53<00:00,  2.26it/s]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0041, Accuracy: 13566/33000 (41.11%)

EPOCH: 2 LR: 0.006500000000000001


Loss=1.6788017749786377 Batch_id=120 Accuracy=60.45: 100%|██████████| 121/121 [00:53<00:00,  2.26it/s]
  0%|          | 0/121 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0043, Accuracy: 13375/33000 (40.53%)

EPOCH: 9 LR: 0.0048000000000000004


Loss=1.431555986404419 Batch_id=120 Accuracy=64.29: 100%|██████████| 121/121 [00:53<00:00,  2.25it/s]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0042, Accuracy: 13489/33000 (40.88%)

EPOCH: 10 LR: 0.0010000000000000009


Loss=1.2244683504104614 Batch_id=120 Accuracy=65.69: 100%|██████████| 121/121 [00:53<00:00,  2.25it/s]
100%|██████████| 52/52 [00:05<00:00,  9.03it/s]


Test set: Average loss: 0.0042, Accuracy: 13591/33000 (41.18%)



In [1]:
model = torchvision.models.resnet18(pretrained=False, num_classes=200).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9,weight_decay = 0.0001,nesterov = True ) 
checkpoint = torch.load("tinymodel.pt")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

model.eval()
print("Loaded the best model")

NameError: ignored

In [21]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=False, threshold=0.0001, threshold_mode='abs', cooldown=0, min_lr=0, eps=1e-08)
train_test.train_model(model,device,trainloader,testloader,optimizer,criterion,10,scheduler,batch_scheduler=False,best_acc = 41.18,path = "tinymodel2.pt")

  0%|          | 0/121 [00:00<?, ?it/s]

EPOCH: 1 LR: 0.0025


Loss=1.5899193286895752 Batch_id=120 Accuracy=60.16: 100%|██████████| 121/121 [00:54<00:00,  2.22it/s]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0041, Accuracy: 13568/33000 (41.12%)

EPOCH: 2 LR: 0.006875000000000001


Loss=1.6240665912628174 Batch_id=120 Accuracy=60.21: 100%|██████████| 121/121 [00:54<00:00,  2.22it/s]
  0%|          | 0/121 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0043, Accuracy: 13308/33000 (40.33%)

EPOCH: 9 LR: 0.005999999999999998


Loss=1.5020170211791992 Batch_id=120 Accuracy=64.34: 100%|██████████| 121/121 [00:54<00:00,  2.22it/s]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0043, Accuracy: 13409/33000 (40.63%)

EPOCH: 10 LR: 0.0024999999999999988


Loss=1.427731990814209 Batch_id=120 Accuracy=65.26: 100%|██████████| 121/121 [00:54<00:00,  2.21it/s]
100%|██████████| 52/52 [00:05<00:00,  8.79it/s]


Test set: Average loss: 0.0042, Accuracy: 13492/33000 (40.88%)

